# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [10]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [11]:
# your code here
orders = pd.read_csv("Orders.csv")

In [12]:
print(orders.head())

   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [15]:
# your code here
#sub problem 1:
customer_spending = orders.groupby("CustomerID")["amount_spent"].sum().reset_index()
print(customer_spending)


      CustomerID  amount_spent
0          12346      77183.60
1          12347       4310.00
2          12348       1797.24
3          12349       1757.55
4          12350        334.40
...          ...           ...
4334       18280        180.60
4335       18281         80.82
4336       18282        178.05
4337       18283       2094.88
4338       18287       1837.28

[4339 rows x 2 columns]


In [ ]:
#Sub problem 2:
# Calculate quantiles
q75 = customer_spending["amount_spent"].quantile(0.75)
q95 = customer_spending["amount_spent"].quantile(0.95)

# Assign labels based on quantile ranges
def label_customer(amount):
    if amount >= q95:
        return "VIP"
    elif amount >= q75:
        return "Preferred"
    else:
        return "Regular"

customer_spending["label"] = customer_spending["amount_spent"].apply(label_customer)

In [ ]:
#Subproblem 3:
print(customer_spending.sort_values("amount_spent", ascending=False).head())

      CustomerID  amount_spent label
1690       14646     280206.02   VIP
4202       18102     259657.30   VIP
3729       17450     194550.79   VIP
3009       16446     168472.50   VIP
1880       14911     143825.06   VIP


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [25]:
# your code here
customer_spending = orders.groupby("CustomerID").agg({
    "amount_spent": "sum",
    "Country": "first"  # Get the first country per customer
}).reset_index()


vip_threshold = customer_spending["amount_spent"].quantile(0.95)
vip_customers = customer_spending[customer_spending["amount_spent"] >= vip_threshold]

vip_by_country = vip_customers["Country"].value_counts()
print(vip_by_country)

top_vip_country = vip_by_country.idxmax()
vip_count = vip_by_country.max()

print(f"Country with the most VIP customers: {top_vip_country} ({vip_count} VIPs)")


Country
United Kingdom     177
Germany             10
France               9
Switzerland          3
Australia            2
EIRE                 2
Portugal             2
Japan                2
Spain                2
Norway               1
Denmark              1
Cyprus               1
Finland              1
Singapore            1
Netherlands          1
Channel Islands      1
Sweden               1
Name: count, dtype: int64
Country with the most VIP customers: United Kingdom (177 VIPs)


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [31]:
# your code here
# Get the first country per customer
customer_spending = orders.groupby("CustomerID").agg({"amount_spent": "sum","Country": "first"}).reset_index()

# Combine VIP and Preferred (but avoid double-counting VIPs)
preferred_only = preferred_customers[~preferred_customers["CustomerID"].isin(vip_customers["CustomerID"])]

# Concatenate both
combined_customers = pd.concat([vip_customers, preferred_only])

# Count per country
combined_by_country = combined_customers["Country"].value_counts()

# Get top country
top_combined_country = combined_by_country.idxmax()
combined_count = combined_by_country.max()

print(f"Country with the most VIP + Preferred customers: {top_combined_country} ({combined_count} customers)")

print(f"Top Country: {top_combined_country}")
print(f"Number of VIP + Preferred Customers: {combined_count}")



Country with the most VIP + Preferred customers: United Kingdom (932 customers)
Top Country: United Kingdom
Number of VIP + Preferred Customers: 932
